In [1]:
from scipy.io import loadmat

path = '/home/zuser/project/WQ/RF_Diffusion/RF-Diffusion/dataset/wifi/cond/user1-1-1-5-5-r5.mat'
data = loadmat(path,verify_compressed_data_integrity=False)

In [2]:
import torch
import torch.nn.functional as F

feature = torch.view_as_real(torch.from_numpy(data['feature']).to(torch.complex64)).permute(1, 2, 0)
down_sample = F.interpolate(feature, 512, mode='nearest-exact')
norm_data = (down_sample - down_sample.mean()) / down_sample.std()
feature = norm_data.permute(2, 0, 1).unsqueeze(0).to('cuda:0')

cond = torch.view_as_real(torch.from_numpy(data['cond']).to(torch.complex64)).to('cuda:0')

cond.shape, feature.shape

(torch.Size([1, 6, 2]), torch.Size([1, 512, 90, 2]))

In [3]:
from tfdiff.wifi_model import tfdiff_WiFi
from tfdiff.params import AttrDict, all_params
from torch.utils.tensorboard import SummaryWriter
from tfdiff.diffusion import SignalDiffusion

writer = SummaryWriter()
params = all_params[0]
batch_size = cond.shape[0]
batch_max = 20*torch.ones(batch_size, dtype=torch.int64)
diffusion = SignalDiffusion(params).to('cuda:0')
x_s = diffusion.degrade_fn(feature, batch_max, 0).to('cuda:0')
model = tfdiff_WiFi(AttrDict(params)).to('cuda:0')
writer.add_graph(model, (x_s, batch_max, cond))
writer.close()

/home/zuser/project/WQ/RF_Diffusion/RF-Diffusion/complex/complex_module.py:278: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  complex_bmm(queries, keys.transpose(1, 2)) / math.sqrt(query_key_dim)


: 

In [14]:
torch.onnx.export(model, (x_s, batch_max, cond), 'tfdiff_wifi.onnx')

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [15]:
import netron
path = 'tfdiff_wifi.onnx'
netron.start(path)

Serving 'tfdiff_wifi.onnx' at http://localhost:8080


('localhost', 8080)